# A demo of the old xpdacq

## Load the helper functions and class

This is done by ipython startup at the beamline.

In [1]:
%run -i 999-load.py

/Users/sst/anaconda3/envs/test_xpdstack/lib/python3.8/site-packages/databroker/v1.py:1602: UserWarning: Failed to load config. Falling back to v0.Exception was: Unable to handle metadatastore.module 'databroker.headersource.sqlite'
  warnings.warn(


No config file could be found in the following locations:
/Users/sst/.config/acq
/Users/sst/anaconda3/envs/test_xpdstack/etc/acq
/etc/acq
Loading from packaged simulation configuration
INFO: Initializing the XPD data acquisition environment ...
INFO: area detector has been configured to new acquisition time (time per frame)  = 0.1s
INFO: Reload beamtime objects:

ScanPlans:
0: ct_5
1: ct_0.1
2: ct_1
3: ct_10
4: ct_30
5: ct_60

Samples:
0: poop

{'Verification time': '2021-05-14 17:38:24', 'Verified by': 'at'}



Is this configuration correct? y/n:  y
Please input your initials:  st


INFO: beamtime object has been linked

INFO: Initialized glbl, bt, xrun.
INFO: Publish data to localhost port 5567 with prefix 'raw'.
INFO: Changed home to /Users/sst/acqsim/xpdUser
OK, ready to go.  To continue, follow the steps in the xpdAcqdocumentation at http://xpdacq.github.io/xpdacq


## The ophyd devices

http://nsls-ii.github.io/#overview-of-software-components

In [5]:
pe1c.describe()

{'pe1_image': {'source': 'SIM:pe1_image',
  'dtype': 'array',
  'shape': [2048, 2048],
  'precision': 3,
  'external': 'FILESTORE'}}

In [6]:
shctl1.describe()

OrderedDict([('rad',
              {'source': 'SIM:rad',
               'dtype': 'integer',
               'shape': [],
               'precision': 3}),
             ('shctl1_setpoint',
              {'source': 'SIM:shctl1_setpoint',
               'dtype': 'integer',
               'shape': [],
               'precision': 3})])

## A demo of a most basic experiment

In [7]:
import bluesky.plan_stubs as bps


def the_simplest_xray_diffraction_measurement():
    # open a run, issue a start document
    yield from bps.open_run()
    # put the detector to active state
    yield from bps.stage(pe1c)
    # open the shutter
    yield from bps.mv(shctl1, 3.14 / 2)
    # trigger and read the image, issue descriptors and events
    sts = yield from bps.trigger_and_read([pe1c], name="primary")
    # close the shutter
    yield from bps.mv(shctl1, 0)
    # put the detector to idle state
    yield from bps.unstage(pe1c)
    # close a run, issue a stop document
    yield from bps.close_run()
    return sts

In [8]:
from bluesky.simulators import summarize_plan


summarize_plan(the_simplest_xray_diffraction_measurement())

=================================== Open Run ===================================
shctl1 -> 1.57
  Read ['pe1_image']
shctl1 -> 0
================================== Close Run ===================================


In [9]:
from pprint import pformat
from bluesky import RunEngine

RE = RunEngine()
RE.subscribe(lambda name, doc: print(name, ':\n', pformat(dict(doc))))

0

In [10]:
RE(the_simplest_xray_diffraction_measurement())

start :
 {'plan_name': 'the_simplest_xray_diffraction_measurement',
 'plan_type': 'generator',
 'scan_id': 1,
 'time': 1619809235.725064,
 'uid': '346c781e-a2a1-494d-ab96-a09180bb5feb',
 'versions': {'bluesky': '1.6.7', 'ophyd': '1.6.1'}}
descriptor :
 {'configuration': {'pe1_image': {'data': {'pe1_image': '4cfde193-6d14-4b8c-bb8a-820d44db9593/0'},
                                 'data_keys': {'pe1_image': {'dtype': 'array',
                                                             'external': 'FILESTORE',
                                                             'precision': 3,
                                                             'shape': [2048,
                                                                       2048],
                                                             'source': 'SIM:pe1_image'}},
                                 'timestamps': {'pe1_image': 1619809235.777209}}},
 'data_keys': {'pe1_image': {'dtype': 'array',
                             'ex

('346c781e-a2a1-494d-ab96-a09180bb5feb',)

## The event model

https://blueskyproject.io/event-model/data-model.html#example-runs

## A helper from xpdacq: bt

`bt` is a `Beamtime` object. It is a python object to store the metadata of samples and plans.

## Create a bt

In [11]:
_start_beamtime?

Signature:
_start_beamtime(
    PI_last,
    saf_num,
    experimenters=[],
    wavelength=None,
    test=False,
)
Docstring: function for start a beamtime
File:      ~/anaconda3/envs/test_xpdstack/lib/python3.8/site-packages/xpdacq/beamtimeSetup.py
Type:      function


In [12]:
bt = _start_beamtime("Billinge", 300000, wavelength=0.167)

INFO: initiated requried directories for experiment
INFO: to link newly created beamtime object to xrun, please do
>>> xrun.beamtime = bt
{'Verification time': '2021-04-30 14:38:53', 'Verified by': 'st'}



Is this configuration correct? y/n:  y
Please input your initials:  st


In [25]:
xrun.beamtime = bt

INFO: beamtime object has been linked



In [13]:
bt.list()


ScanPlans:
0: ct_5
1: ct_0.1
2: ct_1
3: ct_10
4: ct_30
5: ct_60

Samples:


### Add sample to bt

We can add from an excel file or use the helper function

In [2]:
!tree ~/acqsim/xpdUser

/Users/sst/acqsim/xpdUser
├── Import
├── config_base
│   └── yml
│       ├── bt_bt.yml
│       ├── glbl.yml
│       ├── samples
│       └── scanplans
│           ├── ct_0.1.yml
│           ├── ct_1.yml
│           ├── ct_10.yml
│           ├── ct_30.yml
│           ├── ct_5.yml
│           └── ct_60.yml
├── tiff_base
│   └── meta
│       ├── 20210512-151518_0,1s_8d302c.yaml
│       ├── 20210512-151519_s_93dd38.yaml
│       ├── 20210512-151519_s_bb17fa.yaml
│       ├── 20210512-151520_s_a7cc06.yaml
│       ├── 20210512-151521_s_80976e.yaml
│       ├── 20210512-151522_s_08beff.yaml
│       ├── 20210512-151522_s_ea0675.yaml
│       ├── 20210512-151523_s_48d5c8.yaml
│       ├── 20210512-151524_s_77742a.yaml
│       ├── 20210512-151525_s_b4dd4a.yaml
│       ├── 20210512-151525_s_c8c283.yaml
│       ├── 20210512-151526_s_ae0346.yaml
│       ├── 20210512-152323_0,1s_7061b4.yaml
│       ├── 20210512-152323_s_3455e7.yaml
│       ├── 20210512-152324_s_8e5b1a.yaml
│       ├── 20210512-152325_s_4b

In [5]:
from xpdacq.beamtime import load_calibration_md, xpdacq_count

In [6]:
dd = load_calibration_md("/Users/sst/PycharmProjects/pdfstream/tests/test_data/Ni_poni_file.poni")

In [7]:
xrun(0, xpdacq_count([pe1c], calibration_md=dd))

INFO: closing shutter...
INFO: taking dark frame....
INFO: No calibration file found in config_base.
Scan will still keep going on....
INFO: Current filter status
INFO: flt1 : In
INFO: flt2 : Out
INFO: flt3 : Out
INFO: flt4 : Out
dark frame complete, update dark dict
opening shutter...
INFO: No calibration file found in config_base.
Scan will still keep going on....
INFO: Current filter status
INFO: flt1 : In
INFO: flt2 : Out
INFO: flt3 : Out
INFO: flt4 : Out


('681fe435-3777-41bd-8884-de11b3f4c8f8',
 '2e63503d-9b08-4a3f-a2a2-66aeac4653c3')

In [8]:
dd

{'poni_version': 2,
 'detector': 'Perkin',
 'dist': 0.24877604283795038,
 'poni1': 0.2053424178138206,
 'poni2': 0.2009801660742436,
 'rot1': -0.00023710649304047285,
 'rot2': -0.00037576003023777056,
 'rot3': -8.871356199200895e-12,
 'wavelength': 1.671e-11,
 'pixel1': 0.0002,
 'pixel2': 0.0002}

In [3]:
import_sample_info()

INFO: using SAF_number = 300000


FileNotFoundError: no spreadsheet exists in /Users/sst/acqsim/xpdUser/Import
have you put it in with correct naming scheme: '<SAF_num>_sample.xlsx'yet?

In [15]:
bt.list()


ScanPlans:
0: ct_5
1: ct_0.1
2: ct_1
3: ct_10
4: ct_30
5: ct_60

Samples:


In [3]:
Sample(bt, {"sample_name": "poop", "sample_composition": "C"})

Sample({'sample_name': 'poop', 'sample_composition': 'C', 'sa_uid': 'be108e84'}, {'bt_piLast': 'Billinge', 'bt_safN': '300000', 'bt_experimenters': [], 'bt_wavelength': 0.1675, 'bt_uid': '8e7631c8'})

In [17]:
bt.list()


ScanPlans:
0: ct_5
1: ct_0.1
2: ct_1
3: ct_10
4: ct_30
5: ct_60

Samples:
0: poop


## Register a plan in bt

In [15]:
ct?

Signature: ct(dets, exposure)
Docstring:
Take one reading from area detector with given exposure time

Parameters
----------
dets : list
    list of 'readable' objects. default to area detector
    linked to xpdAcq.
exposure : float
    total time of exposrue in seconds

Notes
-----
area detector being triggered will  always be the one configured
in global state. To find out which these are, please using
following commands:

    >>> xpd_configuration['area_det']

to see which device is being linked
File:      ~/anaconda3/envs/test_xpdstack/lib/python3.8/site-packages/xpdacq/beamtime.py
Type:      function


In [19]:
ScanPlan(bt, ct, 35)

ScanPlan({'sp_plan_name': 'ct', 'sp_args': (35,), 'sp_kwargs': {}, 'sp_uid': '448a9634'}, {'bt_piLast': 'Billinge', 'bt_safN': '300000', 'bt_experimenters': [], 'bt_wavelength': 0.167, 'bt_uid': '30b66121'})

In [20]:
bt.list()


ScanPlans:
0: ct_5
1: ct_0.1
2: ct_1
3: ct_10
4: ct_30
5: ct_60
6: ct_35

Samples:
0: poop


## Where are the devices?

In [21]:
xpd_configuration

{'area_det': SynSignalWithRegistry(name='pe1_image', value=array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [-1.699707  ,  4.7998047 , -0.39990234, ...,  1.5       ,
          2.7998047 ,  5.800293  ],
        [ 2.3999023 ,  0.2998047 ,  2.        , ..., -1.6000977 ,
          8.799805  ,  4.5996094 ],
        ...,
        [ 1.5       ,  7.300293  ,  0.09960938, ..., -3.8999023 ,
          5.2998047 , -1.7998047 ],
        [ 0.7998047 ,  3.2998047 ,  3.300293  , ..., 10.        ,
          9.400391  , -1.3999023 ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]], dtype=float32), timestamp=1619809235.777209),
 'shutter': SynAxis(prefix='', name='shctl1', read_attrs=['readback', 'setpoint'], configuration_attrs=['velocity', 'acceleration']),
 'temp_controller': SynAxis(prefix='', name='cs700', read_attrs=['readback', 'setpoint'], configuration_attrs=['velocity', 'acceleration

## A customized RunEngine: xrun

`xrun` is a run engine that can read data from the `bt`. It may also has other subscriber like the database insertion and network publisher.

In [35]:
# run the plan 0 with the metadata of sample 0
xrun(0, 0)

INFO: requested exposure time = 5 - > computed exposure time= 5.0
INFO: This scan will append calibration parameters recorded in /Users/sst/acqsim/xpdUser/config_base/xpdAcq_calib_info.poni
INFO: Current filter status
INFO: flt1 : In
INFO: flt2 : Out
INFO: flt3 : Out
INFO: flt4 : Out


+-----------+------------+
|   seq_num |       time |
+-----------+------------+
|         1 | 16:26:03.8 |
+-----------+------------+
generator count ['e7d6aa9c'] (scan num: 7)




('e7d6aa9c-69e9-414b-9e85-59c988115757',)

In [3]:
import bluesky.plans as bp

In [4]:
xrun(0, bp.scan([pe1c], cs700, 300, 400, 3))

INFO: closing shutter...
INFO: taking dark frame....
INFO: This scan will append calibration parameters recorded in /Users/sst/acqsim/xpdUser/config_base/xpdAcq_calib_info.poni
INFO: Current filter status
INFO: flt1 : In
INFO: flt2 : Out
INFO: flt3 : Out
INFO: flt4 : Out
dark frame complete, update dark dict
opening shutter...
INFO: This scan will append calibration parameters recorded in /Users/sst/acqsim/xpdUser/config_base/xpdAcq_calib_info.poni
INFO: Current filter status
INFO: flt1 : In
INFO: flt2 : Out
INFO: flt3 : Out
INFO: flt4 : Out


('43635799-b5a7-4ed4-88e4-57052792e026',
 'd5cc5063-6112-4222-8368-198d2bd30cf3')

In [5]:
db[-1].metadata["start"]

Start({'analysis_stage': 'raw',
 'beamline_config': {'Verification time': '2021-05-14 16:30:27',
                     'Verified by': 'st'},
 'beamline_id': '28-id-2',
 'bt_experimenters': [],
 'bt_piLast': 'Billinge',
 'bt_safN': '300000',
 'bt_uid': '8e7631c8',
 'bt_wavelength': 0.1675,
 'calibration_md': {'Detector': 'Perkin',
                    'Detector_config': {'pixel1': 0.0002, 'pixel2': 0.0002},
                    'Distance': 0.2243532875271189,
                    'Poni1': 0.20824765844451498,
                    'Poni2': 0.20120937604378425,
                    'Rot1': 0.00697763199117967,
                    'Rot2': 0.00525168715954453,
                    'Rot3': 4.3789540033360746e-10,
                    'Wavelength': 1.675e-11,
                    'poni_version': 2},
 'detector_calibration_client_uid': '6a0aff69-d900-4f55-bde9-67da050e5470',
 'detectors': ['pe1_image'],
 'facility': 'sim',
 'filter_positions': {'flt1': 'In',
                      'flt2': 'Out',
       

In [6]:
from bluesky.callbacks.best_effort import BestEffortCallback

In [7]:
bec = BestEffortCallback()
xrun.subscribe(bec)

3

In [23]:
bec.disable_plots()

In [30]:
from ophyd.sim import hw

In [31]:
HW = hw()

In [35]:
xrun(0, bp.grid_scan([pe1c], cs700, 300, 400, 3, HW.motor1, 300, 400, 3))

INFO: This scan will append calibration parameters recorded in /Users/sst/acqsim/xpdUser/config_base/xpdAcq_calib_info.poni
INFO: Current filter status
INFO: flt1 : In
INFO: flt2 : Out
INFO: flt3 : Out
INFO: flt4 : Out


Transient Scan ID: 16     Time: 2021-05-14 17:30:49
Persistent Unique Scan ID: 'f9e34f7d-d169-4f20-bedb-936a37fe0278'
New stream: 'primary'
+-----------+------------+-------------+------------+
|   seq_num |       time | temperature |     motor1 |
+-----------+------------+-------------+------------+
|         1 | 17:30:49.9 |     300.000 |    300.000 |
|         2 | 17:30:50.7 |     300.000 |    350.000 |
|         3 | 17:30:51.4 |     300.000 |    400.000 |
|         4 | 17:30:52.0 |     350.000 |    300.000 |
|         5 | 17:30:52.6 |     350.000 |    350.000 |
|         6 | 17:30:53.3 |     350.000 |    400.000 |
|         7 | 17:30:53.9 |     400.000 |    300.000 |
|         8 | 17:30:54.6 |     400.000 |    350.000 |
|         9 | 17:30:55.4 |     400.000 |    4

('f9e34f7d-d169-4f20-bedb-936a37fe0278',)

`xrun` can also be used like an usual run engine.

In [36]:
run_calibration()

INFO: requested exposure time = 5 - > computed exposure time= 5.0
INFO: closing shutter...
INFO: taking dark frame....
INFO: This scan will append calibration parameters recorded in /Users/sst/acqsim/xpdUser/config_base/xpdAcq_calib_info.poni
INFO: Current filter status
INFO: flt1 : In
INFO: flt2 : Out
INFO: flt3 : Out
INFO: flt4 : Out


Transient Scan ID: 17     Time: 2021-05-14 17:35:34
Persistent Unique Scan ID: '38a0a295-3a21-4c1b-8f94-32a180b93f0b'
New stream: 'primary'
+-----------+------------+
|   seq_num |       time |
+-----------+------------+


+-----------+------------+
|   seq_num |       time |
+-----------+------------+
|         1 | 17:35:34.4 |
|         1 | 17:35:34.4 |
+-----------+------------+
generator count ['38a0a295'] (scan num: 17)



+-----------+------------+
generator count ['38a0a295'] (scan num: 17)


dark frame complete, update dark dict
opening shutter...
INFO: Current filter status
INFO: flt1 : In
INFO: flt2 : Out
INFO: flt3 : Out
INFO: flt4 : Out


T

KeyboardInterrupt: 

## Run calibration

In [5]:
run_calibration()

INFO: requested exposure time = 5 - > computed exposure time= 5.0
INFO: Current filter status
INFO: flt1 : In
INFO: flt2 : Out
INFO: flt3 : Out
INFO: flt4 : Out


+-----------+------------+
|   seq_num |       time |
+-----------+------------+
|         1 | 15:56:07.4 |
+-----------+------------+
generator count ['722317f1'] (scan num: 3)


INFO: Please navigate to the analysis terminal to complete the interactive calibration process.
You may find the the analysis terminal similar to data acquisition terminal(current terminal) except there is information about the analysis pipeline printed
INFO: For a quick guide on the interactive calibration process, please visit our web-doc at:
https://xpdacq.github.io/xpdAcq/usb_Running.html#calib-manual

Waiting for calibration to finish

If calibration has failed please press Ctrl+C in this terminal and run ``run_calibration`` again!




KeyboardInterrupt: 

## More info

Find more info on https://xpdacq.github.io/xpdAcq/xpdusers.html.